In [1]:
import pandas as pd

# 读取岗位数据
data = pd.read_csv('./data/location_data.csv')

# 去除重复数据
data.drop_duplicates(inplace=True)

# 处理丢失数据
data.fillna('未知', inplace=True)

print(data.head())

   id         company           title    salary education  \
0   0      广东倾云科技有限公司  【初级】web前端开发工程师  2-4K·13薪        大专   
1   1    火眼科技（天津）有限公司         IT运维工程师      2-4K        大专   
2   2  郑州玉带信息技术有限责任公司    实习web前端开发工程师      1-6K        大专   
3   3      广东天勤科技有限公司     兴宁前端、程序员实习生      1-4K        大专   
4   4    武汉赢月网络科技有限公司           运维工程师      2-3K        大专   

                                         description hiring_manager  \
0  1.熟悉HTML5、JavaScript、CSS3;2.熟悉使用css3的flex/grid...            梁先生   
1  一、工作内容1、软件数据分析，月/季报告撰写2、软件平台学习，能够熟练讲解，在使用过程中能够...            刘向宇   
2  参加公司自研产品的Web和微信端小程序开发，使用vue、elementUI和eCharts，...            郭先生   
3  实习生要求：1、需要在梅州兴宁市长期工作（家在梅州市内）2、统招全日制大专或以上学历的应届毕...            林先生   
4                                    负责日常网络及各子系统管理维护             赵爽   

  last_active             address  \
0       2周内活跃      中山广东倾云科技有限公司一层   
1        刚刚活跃       衡水桃城区众成大厦2407   
2        刚刚活跃    郑州二七区升龙世纪花园壹区1号楼   
3          在线        梅州兴宁市天勤网络37号   
4        刚刚活跃  武汉洪山区长

In [3]:
target_jobs = data.query("company == '上海华为技术有限公司' and title == '软件开发工程师'")

# 判断是否发布了两个或更多的软件开发工程师职位
if len(target_jobs) >= 2:
    print("上海华为发布了两个或更多的软件开发工程师职位")
else:
    print("上海华为发布的软件开发工程师职位少于两个")
    
print(target_jobs)

上海华为发布了两个或更多的软件开发工程师职位
        id     company    title      salary education  \
5728  5996  上海华为技术有限公司  软件开发工程师      20-35K        本科   
5737  6006  上海华为技术有限公司  软件开发工程师      15-30K        本科   
5743  6013  上海华为技术有限公司  软件开发工程师  15-30K·15薪        本科   
5962  6232  上海华为技术有限公司  软件开发工程师  35-55K·14薪        本科   
6018  6288  上海华为技术有限公司  软件开发工程师  35-65K·15薪        本科   
7306  7588  上海华为技术有限公司  软件开发工程师  20-40K·14薪        博士   

                                            description hiring_manager  \
5728  在异国获得了丰富经历的你何不在职业生涯中也留下闪光的一笔?或者你觉得壮志难酬，平台不够希望身...            赵先生   
5737  软件开发工程师：岗位职责：1、\t完成从客户需求到软件产品定义、设计、开发、实现，再到上线运...            张先生   
5743  【岗位职责】 1、 负责5G运维相关系统开发，提高团队开发整体研发效率，打造高性能高效工程流...            薛先生   
5962  采用敏捷、Devops、开源等先进的软件设计开发模式，接触最前沿的产品和软件技术，成为大容量...            徐先生   
6018  研究领域：AI框架、操作系统（内核、云和计算，图形图像，嵌入式和车载OS）、编译器与编程语言...            齐女士   
7306  在这里，您将和业界最优秀的软件工程师一起，研发处理性能最优、稳定性最强的产品，您将接触到最先...            张先生   

     last_active                                         a

In [4]:
# 对公司名进行分组，然后计算每个组的大小（即每个公司发布的岗位数量）
job_counts = data.groupby('company').size()

# 显示发布多个岗位的公司
multiple_jobs_companies = job_counts[job_counts > 1]
print(multiple_jobs_companies)

company
\n上海交大四川研究院\n       2
\n中国电子科技集团...\n     2
\n中国航天科工\n          2
\n云软通\n             4
\n伯山羽科技\n           2
                   ..
高通企业管理（上海）有限公司      2
鸿之微科技（上海）股份有限公司     2
鼎捷软件股份有限公司          3
齐齐哈尔市聚合科技有限公司       2
龙岩市新罗区创和智联贸易有限公司    2
Length: 1264, dtype: int64


In [28]:
# 假设你想查询的公司名称为“广东倾云科技有限公司”
company_name = "上海思尔芯技术股份有限公司"

# 使用条件筛选来找到该公司的地址
company_address = data[data['company'] == company_name]['address'].iloc[0]

print(f"{company_name} 的地址是：{company_address}")

上海思尔芯技术股份有限公司 的地址是：深圳南山区国实大厦3F


In [6]:
unique_educations = data['education'].unique()
print(unique_educations)

['大专' '本科' '硕士' '博士']


In [7]:
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer

# 配置jieba分词
jieba.initialize()

# 加载一个更全面的停用词列表，包括中文和英文的常见停用词
with open('./data/stopwords.txt', 'r', encoding='utf-8') as f:
    stop_words = set([line.strip() for line in f.readlines()])

# 分句函数，中文文本中主要以句号、逗号、分号、感叹号等为分隔符
def split_sentences(text):
    separators = ['。', '；', '！', '\n', '，', '.','/']
    sentences = []
    sentence = ''
    for char in text:
        sentence += char
        if char in separators:
            sentences.append(sentence.strip())
            sentence = ''
    if sentence:
        sentences.append(sentence.strip())
    return sentences

# 分词和清洗函数
def tokenize_and_clean(text):
    words = jieba.cut(text, cut_all=False)
    cleaned_words = [word for word in words if word.lower() not in stop_words and len(word) > 1]  # 删除停用词和单字
    return ' '.join(cleaned_words)

# 假设我们已经加载了DataFrame 'data'，且其中包含一个名为'description'的列，存放职位描述
descriptions = data['description'].apply(lambda x: ' '.join([tokenize_and_clean(sentence) for sentence in split_sentences(x)]))

# 初始化TF-IDF向量化器，同时使用一个正则表达式作为token_pattern来更好地处理中英文混合文本
vectorizer = TfidfVectorizer(max_features=1000, token_pattern=r'(?u)\b\w+\b')

# 对清洗后的职位描述进行TF-IDF处理
tfidf_matrix = vectorizer.fit_transform(descriptions)
feature_names = vectorizer.get_feature_names_out()
print(feature_names)
# 接下来，可以使用tfidf_matrix进行聚类分析等后续处理

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\LENOVO\AppData\Local\Temp\jieba.cache
Loading model cost 0.951 seconds.
Prefix dict has been built successfully.


['00' '10' '11' '12' '13' '14' '15' '17' '18' '20' '30' '35' '3d' 'ai'
 'ajax' 'android' 'angular' 'api' 'app' 'arm' 'autosar' 'bgp' 'bim'
 'bootstrap' 'bug' 'c' 'cad' 'can' 'ccie' 'ccnp' 'cisco' 'css' 'css3'
 'c语言' 'design' 'development' 'div' 'dsp' 'eda' 'element' 'erp' 'es6'
 'excel' 'experience' 'fpga' 'git' 'go' 'gulp' 'h3c' 'h5' 'hcie' 'html'
 'html5' 'http' 'i2c' 'ios' 'ip' 'is' 'it' 'java' 'javascript' 'jquery'
 'js' 'json' 'linux' 'matlab' 'mcu' 'mvc' 'mysql' 'net' 'node' 'nodejs'
 'office' 'opencv' 'oracle' 'ospf' 'pc' 'pcb' 'php' 'plc' 'ppt' 'ps'
 'python' 'pytorch' 'qt' 'react' 'redis' 'server' 'shell' 'soc' 'socket'
 'software' 'spi' 'spring' 'sql' 'sqlserver' 'svn' 'tcp' 'team'
 'tensorflow' 'typescript' 'uart' 'ui' 'uni' 'uniapp' 'vpn' 'vue' 'w3c'
 'web' 'webpack' 'windows' 'word' 'work' 'you' '一个' '一体化' '一定' '一年' '一种'
 '一起' '一金' '一门' '三年' '三维' '上位' '上海' '上班时间' '上级' '上级领导' '上线' '上进心' '不同'
 '不定期' '不断' '专业' '专业本科' '专业知识' '专利' '专家' '专科' '业务' '业界' '两年' '严谨' '个人'
 '中心' '中间件' 

In [8]:
print(descriptions[:10])

0     熟悉 HTML5 JavaScript CSS3 熟悉 使用 css3 flex grid...
1    工作 内容 软件 数据分析  报告 撰写 软件平台 学习 能够 熟练 讲解 使用 过程 能够...
2    参加 公司 自研 产品 Web 微信端 程序开发 使用 vue elementUI eCha...
3    实习生 要求 需要 梅州 兴宁市 长期 工作 梅州市 统招 全日制 大专 以上学历 应届 毕...
4                                   负责 日常 网络 子系统 管理 维护
5     专科 以上学历 计算机 通信 电子 相关 专业  热爱 软件开发 工作 移动 产品 浓厚兴...
6    职位 描述 负责 公司 项目 网页 前端 设计 制作 维护 网页 互动 服务 产品 研发 网...
7    任职 要求 精通 HTML CSS JavaScript 良好 编码 习惯 代码 洁癖 优先...
8    岗位职责 负责 公司 相关 web 产品 前端开发 岗位 要求 工作 经验 要求 需要 编码...
9    培养 新人 愿意 学习 勤奋 刻苦 大专 以上学历 游戏 开发 编程 行业 热爱 游戏 开发 大佬
Name: description, dtype: object


In [9]:
from sklearn.cluster import KMeans

# 设定聚类数为 5，这个数字可以根据实际情况调整
n_clusters = 8

# 初始化 KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# 对TF-IDF矩阵进行聚类
kmeans.fit(tfidf_matrix)

# 获取每个职位描述所属的聚类标签
cluster_labels = kmeans.labels_

# 将聚类标签添加到原始数据中，以便分析
data['cluster'] = cluster_labels

# 查看每个聚类的大小
cluster_sizes = data['cluster'].value_counts()

# 打印结果
print(cluster_sizes)

0    1803
3    1349
6    1289
1    1164
4    1014
7     754
5     674
2     415
Name: cluster, dtype: int64


In [10]:
from sklearn.cluster import KMeans

# 设定聚类数目为 8，这个数字可以根据实际情况调整
n_clusters = 8

# 初始化 KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# 对 TF-IDF 矩阵进行转置
tfidf_matrix_transposed = tfidf_matrix.T

# 对转置后的 TF-IDF 矩阵进行聚类
kmeans.fit(tfidf_matrix_transposed)

# 获取每个词汇所属的聚类标签
cluster_labels = kmeans.labels_

# 将聚类标签添加到词汇表中，以便分析
feature_names = vectorizer.get_feature_names_out()
feature_clusters = pd.DataFrame({'word': feature_names, 'cluster': cluster_labels})

# 查看每个聚类的大小
cluster_sizes = feature_clusters['cluster'].value_counts()

# 打印结果
print(cluster_sizes)

5    862
3     62
4     41
1     20
2     10
0      3
6      1
7      1
Name: cluster, dtype: int64


In [11]:
from collections import Counter

# 遍历每个聚类
for cluster_num in range(n_clusters):
    cluster_words = []
    
    # 获取当前聚类中的所有职位描述
    cluster_data = data[data['cluster'] == cluster_num]['description']
    
    # 遍历当前聚类中的每个职位描述
    for description in cluster_data:
        # 将职位描述拆分成单词
        words = description.split()
        cluster_words += words
    
    # 计算当前聚类中每个词汇的出现频率
    word_counts = Counter(cluster_words)
    
    # 打印当前聚类中最常见的词汇
    print(f"Cluster {cluster_num} most common words:")
    print(word_counts.most_common(10))
    print("\n")

Cluster 0 most common words:
[('and', 888), ('to', 374), ('the', 373), ('of', 358), ('in', 344), ('a', 266), ('with', 264), ('for', 183), ('software', 131), ('on', 122)]


Cluster 1 most common words:
[('/', 66), ('Web', 64), ('1.', 53), ('+', 48), ('熟悉', 36), ('2.', 32), ('3.', 30), ('JavaScript', 29), ('React', 28), ('-', 25)]


Cluster 2 most common words:
[('1.', 21), ('3.', 20), ('4.', 17), ('岗位职责：1.', 16), ('2.', 15), ('/', 15), (',', 10), ('5.', 8), ('-', 8), ('或', 7)]


Cluster 3 most common words:
[('/', 63), ('1.', 51), ('岗位职责：1.', 49), ('2.', 33), ('C', 27), ('3.', 25), ('Linux', 23), ('岗位职责：', 22), ('4.', 21), ('1、', 19)]


Cluster 4 most common words:
[('the', 24), ('and', 22), ('岗位职责：', 19), ('?', 17), ('1.', 17), ('岗位职责：1.', 15), ('5.', 15), ('with', 15), ('network', 14), ('of', 13)]


Cluster 5 most common words:
[('and', 125), ('the', 79), ('of', 64), ('?', 46), ('to', 43), ('in', 39), ('1.', 38), ('with', 32), ('3.', 30), ('2.', 28)]


Cluster 6 most common words:
[('

In [12]:
# 为了简化，这里我们仅展示如何打印每个聚类中最常见的词汇
from collections import Counter

for cluster_num in range(n_clusters):
    cluster_words = []
    cluster_data = data[data['cluster'] == cluster_num]['description']
    for description in cluster_data:
        words = description.split()
        cluster_words += words
    most_common_words = Counter(cluster_words).most_common(10)
    print(f"Cluster {cluster_num} most common words:")
    print(most_common_words)
    print("\n")

Cluster 0 most common words:
[('and', 888), ('to', 374), ('the', 373), ('of', 358), ('in', 344), ('a', 266), ('with', 264), ('for', 183), ('software', 131), ('on', 122)]


Cluster 1 most common words:
[('/', 66), ('Web', 64), ('1.', 53), ('+', 48), ('熟悉', 36), ('2.', 32), ('3.', 30), ('JavaScript', 29), ('React', 28), ('-', 25)]


Cluster 2 most common words:
[('1.', 21), ('3.', 20), ('4.', 17), ('岗位职责：1.', 16), ('2.', 15), ('/', 15), (',', 10), ('5.', 8), ('-', 8), ('或', 7)]


Cluster 3 most common words:
[('/', 63), ('1.', 51), ('岗位职责：1.', 49), ('2.', 33), ('C', 27), ('3.', 25), ('Linux', 23), ('岗位职责：', 22), ('4.', 21), ('1、', 19)]


Cluster 4 most common words:
[('the', 24), ('and', 22), ('岗位职责：', 19), ('?', 17), ('1.', 17), ('岗位职责：1.', 15), ('5.', 15), ('with', 15), ('network', 14), ('of', 13)]


Cluster 5 most common words:
[('and', 125), ('the', 79), ('of', 64), ('?', 46), ('to', 43), ('in', 39), ('1.', 38), ('with', 32), ('3.', 30), ('2.', 28)]


Cluster 6 most common words:
[('

In [13]:
cluster_num = 6
cluster_words = []
cluster_data = data[data['cluster'] == cluster_num]['description']
for description in cluster_data:
    words = description.split()
    cluster_words += words
most_common_words = Counter(cluster_words).most_common(50)
print(f"Cluster {cluster_num} most common words:")
print(most_common_words)
print("\n")

Cluster 6 most common words:
[('?', 66), ('1.', 21), ('2.', 20), ('岗位职责：1.', 20), ('5.', 17), ('岗位职责：', 16), ('+', 15), ('任职要求：', 15), ('、', 12), ('1、', 11), ('4.', 11), ('3.', 11), ('1', 9), ('3', 9), ('岗位职责：1、', 8), ('CSS,', 7), ('2', 7), ('and', 7), ('html职位名称：前端开发工程师招聘人数：4学历要求：本科及以上专业要求：计算机或电子通信相关专业毕业任职要求：1.', 6), ('具有扎实的计算机基础知识，熟悉和掌握软件开发的基本概念和流程、有软件开发经验者优先。2.', 6), ('良好的沟通能力，团队意识强，善于发现问题并推动解决问题。3.', 6), ('招聘人数：2人三、?', 6), ('工作方向1、软件：行业软件安装、定制开发和服务外包；2、硬件：语音对讲设备、图像传输设备、网络交换设备、3、音视频会议室系统、卫星通信设备、无线通信基站设备、视频监控安装、弱电综合布线、大屏幕显示系统、广播音响系统的安装调试与运维；4、方案；根据客户需求设计系统集成建设方案。六、?', 6), ('联系方式联系人：杨经理?', 6), ('*****???????', 6), ('王助理?', 6), ('任职资格：', 6), ('4、', 6), ('餐厅销售经理的主要岗位职责', 6), ('4', 6), ('。', 6), ('-', 6), ('技能要求：JavaScript,', 5), ('工作积极主动，具有较强的自主学习能力，吃苦耐劳。', 5), ('五险一金', 5), ('招聘要求：1、计算机或通信专业，专科及以上学历，具备通信技术、信息技术和计算机网络方面的知识，具有熟练的计算机操作技能；2、年龄20~40岁，会开车、有一定工作经验者优先；3', 5), ('工作热情，责任心强，富有创新精神，具有良好的学习能力、分析问题和解决问题的能力；', 5), ('人际沟通能力强，具有良好的团队协作精神。', 5), ('5', 5), ('任职资格：1、有现场造价工作经验者优先；2、熟练掌握相关领域

In [14]:
from collections import Counter
import jieba
# 初始化一个计数器
skills_counter = Counter()

# 假设data中有一列叫'description'，包含职位描述
for description in data['description']:
    # 使用jieba进行中文分词
    words = jieba.cut(description)
    # 更新计数器
    skills_counter.update(words)

# 假设技能词汇长度至少为2个字符，且至少出现了5次
skills_keywords = [skill for skill, count in skills_counter.items() if len(skill) > 2 and count >= 100]

print(skills_keywords)

['HTML5', 'JavaScript', 'CSS3', 'uni', 'app', '模块化', 'Vue', '数据分析', '系统维护', '表达能力', 'PPT', 'CAD', '数据库', '网络设备', '责任心', '实习生', 'Web', '程序开发', 'vue', 'JAVA', '全日制', '以上学历', '毕业生', 'Python', 'VUE', '有限公司', '互联网', '解决方案', '信息技术', '计算机', '软件开发', 'web', 'APP', 'css', 'javascript', 'react', '福利待遇', '节假日', '不定期', 'HTML', 'CSS', '逻辑思维', '实践经验', '岗位职责', '前端开发', 'Javascript', '理工科', '上进心', 'html', '工程师', '高质量', '设计师', '兼容性', '浏览器', '计算机专业', 'html5', 'webpack', '吃苦耐劳', '软件系统', '计算机软件', 'git', '功能模块', '产品开发', '熟练掌握', 'Ajax', 'jQuery', '兼容性问题', '程序设计', '测试计划', '项目经理', '软件工程', '通信工程', '专业本科', 'React', '责任感', '主动性', '基础知识', '应届生', '开发人员', '自动化', 'Angular', '解决问题', '计算机相关', 'JQuery', '人工智能', '认真负责', '管理系统', 'ES6', 'Node', '开发技术', 'WEB', '稳定性', '大学专科', '积极主动', '开发者', 'Git', '开发工具', 'SQL', 'Spring', '设计模式', '软件测试', '测试报告', 'HTTP', '前沿技术', '负责人', 'uniapp', '单元测试', 'W3C', '团队精神', '软硬件', '防火墙', '网络安全', 'design', '试用期', '年终奖', '上班时间', 'PHP', '管理工作', '电路设计', 'BIM', '上级领导', '专业知识', '信息化', '编程语言', '思路清晰', '数据结

In [39]:
import re

# 技能等级关键词和技能个人素质列表
level_keywords = {
    "精通": 4,
    "掌握": 3,
    "熟悉": 2,
    "了解": 1,
}

education_keywords = {
    "博士": 4,
    "硕士": 3,
    "本科": 2,
    "大专": 1, 
}

# 定义技能和个人素质列表
skills_list = [
    "HTML5", "JavaScript", "CSS3", "Vue", "Python", "JAVA", "React", "PHP", "C++", "C语言",
    "uni-app", "jQuery", "Angular", "Bootstrap", "Node.js", "Express", "Vue.js",
    "Git", "Webpack", "MySQL", "Oracle", "SQL Server", "Linux", "Windows", "Android",
    "CAD绘图", "系统集成", "电路设计", "BIM技术", "ERP系统", "控制系统", "电子信息", "通信协议", "智能化", "机器人技术", "控制算法",
    "网络设备", "网络安全", "防火墙", "TCP/IP", "OSPF", "BGP", "VPN", "交换机", "路由器", "网络系统", "网络通信",
    "软件测试", "测试计划", "测试报告", "单元测试", "功能测试", "测试用例", "软件工程", "设计模式", "架构设计", "面向对象设计", "软件开发", "系统维护", "项目管理", "API开发",
    "数据分析", "数据结构", "数据库管理", "信息化", "信息安全", "信息系统", "数据中心", "图像处理", "音视频处理"
]

qualities_list = [
    "表达能力", "PPT制作", "售后服务", "技术支持", "方案设计", 
    "责任心", "上进心", "逻辑思维", "实践经验", "团队精神", "吃苦耐劳", "认真负责", "解决问题", "主动性", "积极性", "沟通", "语言表达"
]

def parse_salary(salary):
    # 提取薪资范围和年薪次数
    match = re.match(r'(\d+)-(\d+)K·(\d+)薪', salary)
    if match:
        low, high, times = map(int, match.groups())
        return (low + high) / 2 * times
    else:
        # 如果没有匹配到年薪次数，假设为12薪
        match = re.match(r'(\d+)-(\d+)K', salary)
        if match:
            low, high = map(int, match.groups())
            return (low + high) / 2 * 12
    # 如果都没有匹配到，返回0
    return 0

def extract_and_rate_skills_and_qualities(description, skills_list, qualities_list):
    skills_ratings = []
    qualities_count = 0
    
    for skill in skills_list:
        for level, rating in level_keywords.items():
            # 使用 re.escape 转义 skill 和 level 中的特殊字符
            escaped_skill = re.escape(skill)
            escaped_level = re.escape(level)
            # 构建正则表达式模式
            pattern = re.compile(f"(?:{escaped_level}\\s*{escaped_skill}|{escaped_skill}\\s*{escaped_level})", re.IGNORECASE)
            if pattern.search(description):
                skills_ratings.append((skill, rating))
                break
    
    # 移除已有的相同技能要求，只保留等级最高的
    skills_ratings = sorted(skills_ratings, key=lambda x: x[1], reverse=True)
    seen_skills = set()
    unique_skills_ratings = []
    for skill, rating in skills_ratings:
        if skill not in seen_skills:
            seen_skills.add(skill)
            unique_skills_ratings.append((skill, rating))
    
    # 如果技能数量大于5，只保留等级最高的5个
    if len(unique_skills_ratings) > 5:
        unique_skills_ratings = unique_skills_ratings[:5]
    
    for quality in qualities_list:
        if quality in description:
            qualities_count += 1
            
    return unique_skills_ratings, qualities_count

# 进行三元组的构建
triples = []
for _, row in data.iterrows():
    company_position = f"{row['company']}_{row['title']}_{row['hiring_manager']}"
    triples.append((row["company"], "公司职位", company_position))  # 添加一个“公司-职位”节点
    triples.append((company_position, "职位", row["title"]))  # “公司-职位”节点与职位节点有关系
    triples.append((company_position, "工作城市", row["city"]))  
    
    # 解析薪资字符串，获取薪资总数
    salary_total = parse_salary(row['salary'])
    triples.append((company_position, "薪资", {"薪资": row["salary"], "总数": salary_total}))  
    
    # 使用education_keywords转换教育要求
    education_score = education_keywords.get(row["education"], 0)  # 默认值为0，如果没有找到匹配项
    triples.append((company_position, "教育要求", education_score))
    triples.append((company_position, "招聘经理", row["hiring_manager"]))
    triples.append((company_position, "活跃时间", row["last_active"]))
    triples.append((company_position, "工作地点", row["address"]))
    triples.append((company_position, "链接", row["link"]))  
    
    skills_ratings, qualities_count = extract_and_rate_skills_and_qualities(row['description'], skills_list, qualities_list)
    
    for skill, rating in skills_ratings:
        triples.append((company_position, "技能要求", {"技能": skill, "等级": rating}))
    if qualities_count > 0:
        triples.append((company_position, "个人素质", qualities_count))       

# 展示部分三元组结果
for subject, predicate, obj, *attributes in triples[0:10]:
    print(f"{subject}\t{predicate}\t{obj}\t{attributes}")

广东倾云科技有限公司	公司职位	广东倾云科技有限公司_【初级】web前端开发工程师_梁先生	[]
广东倾云科技有限公司_【初级】web前端开发工程师_梁先生	职位	【初级】web前端开发工程师	[]
广东倾云科技有限公司_【初级】web前端开发工程师_梁先生	工作城市	中山市	[]
广东倾云科技有限公司_【初级】web前端开发工程师_梁先生	薪资	{'薪资': '2-4K·13薪', '总数': 39.0}	[]
广东倾云科技有限公司_【初级】web前端开发工程师_梁先生	教育要求	1	[]
广东倾云科技有限公司_【初级】web前端开发工程师_梁先生	招聘经理	梁先生	[]
广东倾云科技有限公司_【初级】web前端开发工程师_梁先生	活跃时间	2周内活跃	[]
广东倾云科技有限公司_【初级】web前端开发工程师_梁先生	工作地点	中山广东倾云科技有限公司一层	[]
广东倾云科技有限公司_【初级】web前端开发工程师_梁先生	链接	https://www.zhipin.com/job_detail/2c08c111e75221011n1539S8EVVQ.html?lid=6GPZx3II9Bv.search.31&securityId=f1xZgMo04Kywv-u1CqKcTn5qoCHek_nDfH_HMLOZDR2U7UD6hrtSKuU33BsHCW7ByiFvb2dqc6C5mDevs78kfFXp601IBEUpA0Rr43wDm5o5vDLg5UeiSXzoNHZYpCYCdkPCHw5liZ3l&sessionId=	[]
广东倾云科技有限公司_【初级】web前端开发工程师_梁先生	技能要求	{'技能': 'HTML5', '等级': 2}	[]


In [40]:
triples[:2]

[('广东倾云科技有限公司', '公司职位', '广东倾云科技有限公司_【初级】web前端开发工程师_梁先生'),
 ('广东倾云科技有限公司_【初级】web前端开发工程师_梁先生', '职位', '【初级】web前端开发工程师')]

In [41]:
from py2neo import Graph, Node, Relationship
uri = "bolt://localhost:7687"
user = "neo4j"
password = "mushanmushan"

graph = Graph(uri, auth=(user, password))
result = graph.run("MATCH (n:Salary {name:'1-2K'}) RETURN n")
print(result)

(No data)


In [42]:
from py2neo import Graph, Node, Relationship

# Neo4j 连接配置
uri = "bolt://localhost:7687"  # 修改为您的 Neo4j 实例地址
username = "neo4j"  # 修改为您的用户名
password = "mushanmushan"  # 修改为您的密码

# 初始化 Neo4j 连接
graph = Graph(uri, auth=(username, password))

def create_node_and_relationships(triples):
    created_nodes = {
        "Company": {}, "Position": {}, "CompanyPosition": {}, "Skill": {}, "Education": {},
        "Salary": {}, "Quality": {}, "ActiveTime": {}, "City": {}, "Address": {}
    }

    for subject, predicate, object, *attributes in triples:
        if predicate == "公司职位":
            # 处理公司和“公司-职位”的关系
            company_node = created_nodes["Company"].setdefault(subject, Node("Company", name=subject))
            graph.merge(company_node, "Company", "name")

            company_position_node = created_nodes["CompanyPosition"].setdefault(object, Node("CompanyPosition", id=object))
            graph.merge(company_position_node, "CompanyPosition", "id")

            relationship = Relationship(company_node, "HAS", company_position_node)
            graph.merge(relationship)

        elif predicate == "职位":
            # 处理“公司-职位”和职位的关系
            company_position_node = created_nodes["CompanyPosition"][subject]
            position_node = created_nodes["Position"].setdefault(object, Node("Position", name=object))
            graph.merge(position_node, "Position", "name")

            relationship = Relationship(company_position_node, "POSITION", position_node)
            graph.merge(relationship)

        elif predicate == "技能要求":
            # 特殊处理技能要求，技能等级作为关系属性
            company_position_node = created_nodes["CompanyPosition"][subject]
            skill_node = created_nodes["Skill"].setdefault(object['技能'], Node("Skill", name=object['技能']))
            graph.merge(skill_node, "Skill", "name")

            # 建立关系，并设置技能等级属性
            rel = Relationship(company_position_node, "REQUIRES_SKILL", skill_node, level=object['等级'])
            graph.create(rel)

        elif predicate == "薪资":
            # 创建薪资节点并建立关系，薪资的总数作为关系的level，描述作为节点
            company_position_node = created_nodes["CompanyPosition"][subject]
            salary_node = created_nodes["Salary"].setdefault(object['薪资'], Node("Salary", name=object['薪资']))
            graph.merge(salary_node, "Salary", "name")

            relationship = Relationship(company_position_node, "SALARY", salary_node, level=object['总数'])
            graph.create(relationship)

        elif predicate in ["教育要求", "个人素质", "活跃时间", "工作城市", "工作地点"]:
            # 创建独立节点并建立关系
            company_position_node = created_nodes["CompanyPosition"][subject]
            # 将中文键映射到英文键
            predicate_mapping = {"教育要求": "Education", "个人素质": "Quality", "活跃时间": "ActiveTime", "工作城市": "City", "工作地点": "Address"}
            english_predicate = predicate_mapping[predicate]

            attribute_node = created_nodes[english_predicate].setdefault(object, Node(english_predicate, name=object))
            graph.merge(attribute_node, english_predicate, "name")

            relationship = Relationship(company_position_node, english_predicate.upper(), attribute_node)
            graph.create(relationship)

        elif predicate == "链接":
            # 链接作为公司-职位的属性
            company_position_node = created_nodes["CompanyPosition"][subject]
            company_position_node['链接'] = object
            graph.push(company_position_node)

# 删除所有现有的数据
graph.run("MATCH (n) DETACH DELETE n")

# 导入三元组数据到 Neo4j
create_node_and_relationships(triples)

In [1]:
from py2neo import Graph, Node, Relationship

# Neo4j 连接配置
uri = "bolt://localhost:7687"  # Neo4j 实例地址
username = "neo4j"  # 用户名
password = "mushanmushan"  # 密码

# 初始化 Neo4j 连接
graph = Graph(uri, auth=(username, password))

query = """
MATCH (p:Position {name: '运维工程师'})-[:POSITION]-(cp:CompanyPosition) 
RETURN cp, p LIMIT 5
"""

results = graph.run(query).to_data_frame()
for index, row in results.iterrows():
    print(row["p"]["name"])
    

运维工程师
运维工程师
运维工程师
运维工程师
运维工程师


In [1]:
name_translate={
    "姓名": "name",
    "学历": "education",
    "邮箱":"email",
    "技能":"skills",
    "技能等级":"skills_level",  
    "专业":"major",
    "电话号码":"phone",
    "个人素质":"personality"
}

for label in name_translate.values():
    print(label)

name
education
email
skills
skills_level
major
phone
personality


In [18]:
from py2neo import Graph



# 初始化图数据库连接
graph = Graph("bolt://localhost:7687", auth=("neo4j", "mushanmushan"))

# 参数设置
params = {
    "city": "北京市",
    "education_levels": [1, 2],
    "limit": 20
}

# 查询语句
query = """
MATCH (cn:Company)-[:HAS]->(cp:CompanyPosition)-[:POSITION]->(p:Position),
      (cp)-[:CITY]->(c:City),
      (cp)-[:EDUCATION]->(e:Education),
      (cp)-[:REQUIRES_SKILL]->(sk:Skill),
      (cp)-[:SALARY]->(s:Salary)
WHERE c.name = $city
  AND e.name IN $education_levels
WITH p, cn, cp, s, e, collect(sk.name) AS matchedSkills, count(sk) AS skillMatchCount
RETURN p.name AS Position,
       cn.name AS Company,
       s.name AS Salary,
       e.name AS Education,
       matchedSkills,
       skillMatchCount
ORDER BY skillMatchCount DESC
LIMIT $limit
"""

# 执行查询
results = graph.run(query, **params)

# 输出结果
for record in results:
    print(dict(record))


{'Position': '前端开发工程师', 'Company': '北京医合易生科技有限公司', 'Salary': '15-25K', 'Education': 1, 'matchedSkills': ['HTML5', 'Vue.js', 'JAVA', 'Vue', 'JavaScript'], 'skillMatchCount': 5}
{'Position': '前端开发工程师 （react方向）(J10401)', 'Company': '北京九恒星科技股份有限公司', 'Salary': '15-25K', 'Education': 1, 'matchedSkills': ['Webpack', 'React', 'JAVA', 'JavaScript'], 'skillMatchCount': 4}
{'Position': '应用软件工程师', 'Company': '北京网讯鹏博科技有限公司', 'Salary': '10-12K', 'Education': 2, 'matchedSkills': ['Windows', 'Linux', 'JAVA', 'Python'], 'skillMatchCount': 4}
{'Position': '前端开发工程师', 'Company': '神州通誉软件（上海）股份有限公司', 'Salary': '15-21K', 'Education': 1, 'matchedSkills': ['Webpack', 'Git', 'Vue', 'CSS3'], 'skillMatchCount': 4}
{'Position': 'web前端开发工程师', 'Company': '妈妈在线国际网络科技有限公司北京分公司', 'Salary': '14-28K', 'Education': 1, 'matchedSkills': ['PHP', 'HTML5', 'JAVA', 'JavaScript'], 'skillMatchCount': 4}
{'Position': '三维前端开发工程师', 'Company': '北京六合神州科技有限公司', 'Salary': '18-35K', 'Education': 1, 'matchedSkills': ['React', 'JAVA', 'Jav